In [14]:
import numpy as np
#import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as v

import Net_and_Loss as nl

In [15]:
is_gpu = torch.cuda.is_available()
if is_gpu:
    id = 1
    torch.cuda.set_device(id)
    
#gpu_nums = torch.cuda.device_count()
#gpu_index = torch.cuda.current_device()
#print(is_gpu,gpu_nums,gpu_index)
device = torch.device('cuda' if is_gpu else 'cpu')

#device = torch.device('cpu')
print(device)
torch.set_default_tensor_type('torch.DoubleTensor')
print(torch.__version__)

cuda
1.6.0+cu101


In [57]:
k , theta0 = 4 , np.pi/7
k1,k2 = k*np.cos(theta0) , k*np.sin(theta0) 
sample_num = 800 
k_2 = k*k
theta = torch.linspace(2*np.pi/sample_num,2*np.pi,sample_num).reshape(-1,1)
r = 9/20-1/9*torch.cos(5*theta)
x0,x1 = torch.cos(theta),torch.sin(theta)
sample_x = v(torch.cat((x0,x1),1)*r,requires_grad=True)

sample_u_r = torch.cos(k1*sample_x[:,0]+k2*sample_x[:,1]); 
sample_u_i = torch.sin(k1*sample_x[:,0]+k2*sample_x[:,1]);

sample = 2400
theta_in = torch.rand(sample,1)*2*np.pi
r_in = torch.rand(sample,1)*(9/20-1/9*torch.cos(5*theta_in))*0.999
x0_in = torch.cos(theta_in)
x1_in = torch.sin(theta_in)
x_in = v(torch.cat((x0_in,x1_in),1)*r_in,requires_grad = True)

class myNet1(nn.Module):
  def __init__(self,m,out):
    super().__init__()

    self.net=nn.Sequential(
      nn.Linear(in_features=2,out_features=m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,2)
    )
  def forward(self, input:torch.FloatTensor):
    return self.net(input)
class Pinn_loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,ddu_r,ddu_i,u_r,u_i,bd_out_r,bd_out_i,bd_u_r,bd_u_i,k2):
        return torch.mean((ddu_r+k2*u_r)**2+(ddu_i+k2*u_i)**2)+1*torch.mean((bd_out_r-bd_u_r)**2+(bd_out_i-bd_u_i)**2)
    
mynet = myNet1(40,2)


In [58]:
mynet = mynet.to(device)
x_in = x_in.to(device)
sample_x = sample_x.to(device)
sample_u_r,sample_u_i = sample_u_r.to(device),sample_u_i.to(device) 
out_r = mynet(x_in)[:,0]
c = (torch.ones(out_r.size())).to(device)
out_r,out_i = mynet(x_in)[:,0],mynet(x_in)[:,1]
bd_out_r,bd_out_i = mynet(sample_x)[:,0],mynet(sample_x)[:,1]
dx_r = torch.autograd.grad(out_r, x_in, grad_outputs=c,retain_graph=True,create_graph=True)[0]
grad_x_r = (dx_r[:,0]).reshape(-1,1)
c0 = (torch.ones(grad_x_r.size())).to(device)

In [59]:
optimzer = torch.optim.Adam(mynet.parameters(mynet),lr=0.0001)
loss_func = Pinn_loss()
epoch = 20000
loss_all = np.zeros(epoch+1)
for epoc in range(epoch+1):
    out_r,out_i = mynet(x_in)[:,0],mynet(x_in)[:,1]
    bd_out_r,bd_out_i = mynet(sample_x)[:,0],mynet(sample_x)[:,1]
    
    dx_r = torch.autograd.grad(out_r, x_in, grad_outputs=torch.ones(out_r.size()).to(device),retain_graph=True,create_graph=True)[0]
    grad_x_r = (dx_r[:,0]).reshape(-1,1)
    grad_y_r = (dx_r[:,1]).reshape(-1,1)
    dxx_r = torch.autograd.grad(grad_x_r, x_in, grad_outputs=torch.ones(grad_x_r.size()).to(device),retain_graph=True,create_graph=True)[0]
    dyy_r = torch.autograd.grad(grad_y_r, x_in, grad_outputs=torch.ones(grad_y_r.size()).to(device),retain_graph=True,create_graph=True)[0]
    duu_r = (dxx_r[:,0]).reshape(-1,1)+(dyy_r[:,1]).reshape(-1,1)
    
    dx_i = torch.autograd.grad(out_i, x_in, grad_outputs=torch.ones(out_i.size()).to(device),retain_graph=True,create_graph=True)[0]
    grad_x_i = (dx_i[:,0]).reshape(-1,1)
    grad_y_i = (dx_i[:,1]).reshape(-1,1)
    dxx_i = torch.autograd.grad(grad_x_i, x_in, grad_outputs=torch.ones(grad_x_i.size()).to(device),retain_graph=True,create_graph=True)[0]
    dyy_i = torch.autograd.grad(grad_y_i, x_in, grad_outputs=torch.ones(grad_y_i.size()).to(device),retain_graph=True,create_graph=True)[0]
    duu_i = (dxx_i[:,0]).reshape(-1,1)+(dyy_i[:,1]).reshape(-1,1)
    
    loss = loss_func(duu_r,duu_i,out_r,out_i,bd_out_r,bd_out_i,sample_u_r,sample_u_i,k_2)
    optimzer.zero_grad()
    loss.backward(retain_graph=True)
    optimzer.step()
    if epoc%1000==0:
        print(loss,epoc,torch.mean((bd_out_r-sample_u_r)**2+(sample_u_i-bd_out_i)**2))
        
    #loss_all[epoc]=loss.cpu().detach().numpy()

tensor(11.4053, device='cuda:1', grad_fn=<AddBackward0>) 0 tensor(0.9655, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9995, device='cuda:1', grad_fn=<AddBackward0>) 1000 tensor(0.9991, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9995, device='cuda:1', grad_fn=<AddBackward0>) 2000 tensor(0.9990, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9993, device='cuda:1', grad_fn=<AddBackward0>) 3000 tensor(0.9989, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9986, device='cuda:1', grad_fn=<AddBackward0>) 4000 tensor(0.9981, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9943, device='cuda:1', grad_fn=<AddBackward0>) 5000 tensor(0.9909, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9920, device='cuda:1', grad_fn=<AddBackward0>) 6000 tensor(0.9852, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9910, device='cuda:1', grad_fn=<AddBackward0>) 7000 tensor(0.9829, device='cuda:1', grad_fn=<MeanBackward0>)
tensor(0.9878, device='cuda:1', grad_fn=<AddBackward0>) 80

In [56]:
#sample = 1000
#theta_in = torch.rand(sample,1)*2*np.pi
#r_in = torch.rand(sample,1).sqrt()*(9/20-1/9*torch.cos(5*theta_in))*0.99
#x0_in = torch.cos(theta_in)
#x1_in = torch.sin(theta_in)
#x_in = (torch.cat((x0_in,x1_in),1)*r_in).to(device)
out_r,out_i = mynet(x_in)[:,0],mynet(x_in)[:,1]
u_in_r = (torch.cos(k1*x_in[:,0]+k2*x_in[:,1]))#.to(device)
u_in_i = (torch.sin(k1*x_in[:,0]+k2*x_in[:,1]))#.to(device)
#print(out_r)
#print(u_in_r)
#print(out_i)
#print(u_in_i)
print((((out_r-u_in_r)**2+(out_i-u_in_i)**2).sum()/((u_in_r)**2+(u_in_i)**2).sum()).sqrt())

tensor(0.9852, device='cuda:1', grad_fn=<SqrtBackward>)
